In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit


spark = SparkSession.builder.appName("testing").enableHiveSupport().getOrCreate()


In [9]:
spark.sql("CREATE DATABASE IF NOT EXISTS test")
spark.sql("DROP TABLE IF EXISTS test.sampleTable")
spark.sql("""
CREATE TABLE test.sampleTable (
    _change_type string,
    _commit_version bigint,
    _commit_timestamp timestamp,
    __Timestamp bigint,
    __DeleteVersion timestamp,
    __UpsertVersion timestamp,
    __DROP_EXPECTATIONS_COL map<string,boolean>,
    __MEETS_DROP_EXPECTATIONS boolean,
    __ALLOW_EXPECTATIONS_COL map<string,boolean>
)
""")
spark.sql("DESCRIBE TABLE test.sampleTable").show()


23/11/17 13:57:39 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
23/11/17 13:57:39 WARN HiveMetaStore: Location: file:/Users/williamhughes/code/scratch_pad/spark-warehouse/test.db/sampletable specified for non-external table:sampletable
+--------------------+-------------------+-------+
|            col_name|          data_type|comment|
+--------------------+-------------------+-------+
|        _change_type|             string|   null|
|     _commit_version|             bigint|   null|
|   _commit_timestamp|          timestamp|   null|
|         __Timestamp|             bigint|   null|
|     __DeleteVersion|          timestamp|   null|
|     __UpsertVersion|          timestamp|   null|
|__DROP_EXPECTATIO...|map<string,boolean>|   null|
|__MEETS_DROP_EXPE...|            boolean|   null|
|__ALLOW_EXPECTATI...|ma

In [13]:
rows = spark.sql("DESCRIBE TABLE test.sampleTable").collect()
new_columns = {row.col_name: lit(None).cast(row.data_type) for row in rows if row.col_name.startswith("_")}
df = spark.createDataFrame([(2,)], schema=["test"])
df.withColumns(new_columns).dtypes


[('test', 'bigint'),
 ('_change_type', 'string'),
 ('_commit_version', 'bigint'),
 ('_commit_timestamp', 'timestamp'),
 ('__Timestamp', 'bigint'),
 ('__DeleteVersion', 'timestamp'),
 ('__UpsertVersion', 'timestamp'),
 ('__DROP_EXPECTATIONS_COL', 'map<string,boolean>'),
 ('__MEETS_DROP_EXPECTATIONS', 'boolean'),
 ('__ALLOW_EXPECTATIONS_COL', 'map<string,boolean>')]